In [1]:
## Librerías
import numpy as np
import seaborn as sns
import sys
import cv2 as cv2
import glob
from PIL import Image
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import gc
import keras
from sklearn.model_selection import train_test_split
import os

import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers
tfd = tfp.distributions

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import umap
from sklearn.preprocessing import StandardScaler

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import glob

import random 

2023-07-29 04:24:19.126169: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-29 04:24:20.990229: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mo

In [ ]:
## Uso de GPU
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

## Cargar dataset de micrografías secas

In [37]:
### Rutas de todas las imágenes

## Obtener todas las rutas de las imágenes en el dataset
ruta_dataset  = glob.glob('/home/willy98/archivos/Datasets/SH_procesadas/Recortadas_grises/*/*/*/*.jpg')

## Capturar solo rutas secas 
rutas_imagenes_secas = []
rutas_imagenes_humedas = []

clases = []
for ruta in ruta_dataset:
    if ruta.split('/')[-2] == 'Secas' or ruta.split('/')[-2] == 'Seca':
        clases.append(ruta.split('/')[-4])
        rutas_imagenes_secas.append(ruta)
    else:
        rutas_imagenes_humedas.append(ruta)
    
print(np.shape(rutas_imagenes_secas),np.shape(rutas_imagenes_humedas))
print(np.unique(clases))

(931,) (792,)
['CuNi1' 'CuNi2' 'CuNi3']


In [38]:
data_secas = pd.DataFrame({'ruta':rutas_imagenes_secas,'clase':clases})



In [39]:
data_secas

,ruta,clase
0,/home/willy98/archivos/Datasets/SH_procesadas/...,CuNi1
1,/home/willy98/archivos/Datasets/SH_procesadas/...,CuNi1
2,/home/willy98/archivos/Datasets/SH_procesadas/...,CuNi1
3,/home/willy98/archivos/Datasets/SH_procesadas/...,CuNi1
4,/home/willy98/archivos/Datasets/SH_procesadas/...,CuNi1
...,...,...
926,/home/willy98/archivos/Datasets/SH_procesadas/...,CuNi2
927,/home/willy98/archivos/Datasets/SH_procesadas/...,CuNi2
928,/home/willy98/archivos/Datasets/SH_procesadas/...,CuNi2
929,/home/willy98/archivos/Datasets/SH_procesadas/...,CuNi2
